# Feature Engineering

The previous sections outline the fundamental ideas of machine learning, but all of the examples assume that you have numerical data in a tidy, ``[n_samples, n_features]`` format.
In the real world, data rarely comes in such a form.
With this in mind, one of the more important steps in using machine learning in practice is *feature engineering*: that is, taking whatever information you have about your problem and turning it into numbers that you can use to build your feature matrix.

In this section, we will cover a few common examples of feature engineering tasks: features for representing *categorical data*, features for representing *text*, and features for representing *images*.
Additionally, we will discuss *derived features* for increasing model complexity and *imputation* of missing data.
Often this process is known as *vectorization*, as it involves converting arbitrary data into well-behaved vectors.

## Categorical Features

One common type of non-numerical data is *categorical* data.
For example, imagine you are exploring some data on housing prices, and along with numerical features like "price" and "rooms", you also have "neighborhood" information.
For example, your data might look something like this:

In [ ]:
data = [
    {'price': 850000, 'rooms': 4, 'neighborhood': 'Queen Anne'},
    {'price': 700000, 'rooms': 3, 'neighborhood': 'Fremont'},
    {'price': 650000, 'rooms': 3, 'neighborhood': 'Wallingford'},
    {'price': 600000, 'rooms': 2, 'neighborhood': 'Fremont'}
]

You might be tempted to encode this data with a straightforward numerical mapping:

In [ ]:
{'Queen Anne': 1, 'Fremont': 2, 'Wallingford': 3};

It turns out that this is not generally a useful approach in Scikit-Learn: the package's models make the fundamental assumption that numerical features reflect algebraic quantities.
Thus such a mapping would imply, for example, that *Queen Anne < Fremont < Wallingford*, or even that *Wallingford - Queen Anne = Fremont*, which (niche demographic jokes aside) does not make much sense.

In this case, one proven technique is to use *one-hot encoding*, which effectively creates extra columns indicating the presence or absence of a category with a value of 1 or 0, respectively.
When your data comes as a list of dictionaries, Scikit-Learn's ``DictVectorizer`` will do this for you:

In [ ]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False, dtype=int)
vec.fit_transform(data)

Notice that the 'neighborhood' column has been expanded into three separate columns, representing the three neighborhood labels, and that each row has a 1 in the column associated with its neighborhood.
With these categorical features thus encoded, you can proceed as normal with fitting a Scikit-Learn model.

To see the meaning of each column, you can inspect the feature names:

In [ ]:
vec.get_feature_names()

There is one clear disadvantage of this approach: if your category has many possible values, this can *greatly* increase the size of your dataset.
However, because the encoded data contains mostly zeros, a sparse output can be a very efficient solution:

In [ ]:
vec = DictVectorizer(sparse=True, dtype=int)
vec.fit_transform(data)

In [ ]:
vec.feature_names_

## sklearn.preprocessing: OneHotEncoder

Encode categorical integer features using a one-hot aka one-of-K scheme.

In [ ]:
from sklearn import preprocessing
import pandas as pd

titanic = {'name': ['Owen Harris Braund','John Bradley Cuming','Laina Heikkinen'],
           'gender':['male', 'male', 'female'],
           'ticket':['A/5 21171','PC 17599','STON/O2. 3101282'],
           'cabin':['','C85',''],
           'embarked':['S','C','S']}

df = pd.DataFrame(titanic)
enc = preprocessing.LabelEncoder()

df['name']=enc.fit_transform(df['name'])
df

In [ ]:
from sklearn import preprocessing
import pandas as pd
from collections import defaultdict

d = defaultdict(preprocessing.LabelEncoder)

titanic = {'name': ['Owen Harris Braund','John Bradley Cuming','Laina Heikkinen'],
           'gender':['male', 'male', 'female'],
           'ticket':['A/5 21171','PC 17599','STON/O2. 3101282'],
           'cabin':['','C85',''],
           'embarked':['S','C','S']}

df = pd.DataFrame(titanic)

# Encoding the variable
fit = df.apply(lambda x: d[x.name].fit_transform(x))
fit

In [ ]:
# Inverse the encoded
df_new=fit.apply(lambda x: d[x.name].inverse_transform(x))
df_new

## Pandas

Pandas has ``get_dummies()`` function that able to convert categorical variables to dummy variables

In [ ]:
import pandas as pd
import numpy as np
my_data = np.array([[5, 'a', 1],
                    [3, 'b', 3],
                    [1, 'b', 2],
                    [3, 'a', 1],
                    [4, 'b', 2],
                    [7, 'c', 1],
                    [7, 'c', 1]])                
df = pd.DataFrame(data=my_data, columns=['y', 'dummy', 'x'])
just_dummy = pd.get_dummies(df['dummy'])
df_1 = pd.concat([df, just_dummy], axis=1)      
df_1.drop(['dummy', 'c'], inplace=True, axis=1)
df_1

### Exercise 1

Encode the gender using the one-hot encoding technique.

```{python}
raw_data = {'first_name': ['Jason', 'Tina', 'Rish', 'Wesley ', 'Amy'],
        'last_name': ['Phoon', 'Lau', 'Richer', 'Lee', 'Lim'],
        'gender': ['male', 'female', 'male', 'male', 'female']}
```

In [ ]:
from sklearn import preprocessing
import pandas as pd

raw_data = {'first_name': ['Jason', 'Tina', 'Rish', 'Wesley ', 'Amy'],
        'last_name': ['Phoon', 'Lau', 'Richer', 'Lee', 'Lim'],
        'gender': ['male', 'female', 'male', 'male', 'female']}

df = pd.DataFrame(raw_data)
enc = preprocessing.LabelEncoder()
print(df)
enc.fit_transform(df['gender'])

Many (though not yet all) of the Scikit-Learn estimators accept such sparse inputs when fitting and evaluating models. ``sklearn.preprocessing.OneHotEncoder`` and ``sklearn.feature_extraction.FeatureHasher`` are two additional tools that Scikit-Learn includes to support this type of encoding.

### LabelEncoder

Encode labels with value between 0 and n_classes-1. LabelEncoder assigns ordinal levels to categorical data. 

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train = ["Selangor", "Selangor", "Wilayah Persekutuan", "Kedah"]
test = ["Wilayah Persekutuan", "Wilayah Persekutuan", "Selangor"]

le.fit(train)

#print the encoded label
print(le.classes_)

#print the label for Test
le.transform(test)


In [ ]:
#print back the actual data values
le.inverse_transform([1,1,0,2])

### Exercise 2

Load the iris dataset and use the ``LabelEncoder`` to encode the 'Species' target variabe

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

print(iris.target_names)

enc = preprocessing.LabelEncoder()

enc.fit_transform(iris.target_names)

## Text Features

Another common need in feature engineering is to convert text to a set of representative numerical values.
For example, most automatic mining of social media data relies on some form of encoding the text as numbers.
One of the simplest methods of encoding data is by *word counts*: you take each snippet of text, count the occurrences of each word within it, and put the results in a table.

For example, consider the following set of three phrases:

In [ ]:
sample = ['problem of evil',
          'evil queen',
          'horizon problem']

For a vectorization of this data based on word count, we could construct a column representing the word "problem," the word "evil," the word "horizon," and so on.
While doing this by hand would be possible, the tedium can be avoided by using Scikit-Learn's ``CountVectorizer``:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(sample)

The result is a sparse matrix recording the number of times each word appears; it is easier to inspect if we convert this to a ``DataFrame`` with labeled columns:

In [ ]:
import pandas as pd
pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

There are some issues with this approach, however: the raw word counts lead to features which put too much weight on words that appear very frequently, and this can be sub-optimal in some classification algorithms.
One approach to fix this is known as *term frequency-inverse document frequency* (*TF–IDF*) which weights the word counts by a measure of how often they appear in the documents.
The syntax for computing these features is similar to the previous example:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X = vec.fit_transform(sample)
pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

## Image Features

Another common need is to suitably encode *images* for machine learning analysis.
The simplest approach is what we used for the handwritten digits recognition by simply using the pixel values themselves.

But depending on the application, such approaches may not be optimal.

A comprehensive summary of feature extraction techniques for images is well beyond the scope of this section, but you can find excellent implementations of many of the standard approaches in the [Scikit-Image project](http://scikit-image.org).

## Derived Features

Another useful type of feature is one that is mathematically derived from some input features.
We saw an example of this in **Hyperparameters and Model Validation** when we constructed *polynomial features* from our input data.

We saw that we could convert a linear regression into a polynomial regression not by changing the model, but by transforming the input!
This is sometimes known as *basis function regression*.

For example, this data clearly cannot be well described by a straight line:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

x = np.array([1, 2, 3, 4, 5])
y = np.array([4, 2, 1, 3, 7])
plt.scatter(x, y);

Still, we can fit a line to the data using ``LinearRegression`` and get the optimal result:

In [ ]:
from sklearn.linear_model import LinearRegression
X = x[:, np.newaxis]
model = LinearRegression().fit(X, y)
yfit = model.predict(X)
plt.scatter(x, y)
plt.plot(x, yfit);

It's clear that we need a more sophisticated model to describe the relationship between $x$ and $y$.

One approach to this is to transform the data, adding extra columns of features to drive more flexibility in the model.
For example, we can add polynomial features to the data this way:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, include_bias=False)
X2 = poly.fit_transform(X)
print(X2)

The derived feature matrix has one column representing $x$, and a second column representing $x^2$, and a third column representing $x^3$.
Computing a linear regression on this expanded input gives a much closer fit to our data:

In [ ]:
model = LinearRegression().fit(X2, y)
yfit = model.predict(X2)
plt.scatter(x, y)
plt.plot(x, yfit);

This idea of improving a model not by changing the model, but by transforming the inputs, is fundamental to many of the more powerful machine learning methods.

## Imputation of Missing Data

Another common need in feature engineering is handling of missing data.
We discussed the handling of missing data in ``DataFrame``s and saw that often the ``NaN`` value is used to mark missing values.

For example, we might have a dataset that looks like this:

In [ ]:
from numpy import nan
X = np.array([[ nan, 0,   3  ],
              [ 3,   7,   9  ],
              [ 3,   5,   2  ],
              [ 4,   nan, 6  ],
              [ 8,   8,   1  ]])
y = np.array([14, 16, -1,  8, -5])

When applying a typical machine learning model to such data, we will need to first replace such missing data with some appropriate fill value.
This is known as *imputation* of missing values, and strategies range from simple (e.g., replacing missing values with the mean of the column) to sophisticated (e.g., using matrix completion or a robust model to handle such data).

The sophisticated approaches tend to be very application-specific, and we won't dive into them here.
For a baseline imputation approach, using the mean, median, or most frequent value, Scikit-Learn provides the ``Imputer`` class:

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='mean')
X2 = imp.fit_transform(X)
X2

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='most_frequent')
X2 = imp.fit_transform(X)
X2

We see that in the resulting data, the two missing values have been replaced with the mean of the remaining values in the column. This imputed data can then be fed directly into, for example, a ``LinearRegression`` estimator:

In [ ]:
model = LinearRegression().fit(X2, y)
model.predict(X2)

## Pandas: fillna

Fill NA/NaN values using the specified method. For more information, read the [documentation](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.fillna.html)

In [ ]:
import pandas as pd

raw_data = {'first_name': ['Jason', np.nan, 'Tina', 'Jake', 'Amy'],
        'last_name': ['Miller', np.nan, 'Ali', 'Milner', 'Cooze'],
        'age': [42, np.nan, 36, 24, 73],
        'sex': ['m', np.nan, 'f', 'm', 'f'],
        'preTestScore': [4, np.nan, np.nan, 2, 3],
        'postTestScore': [25, np.nan, np.nan, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'sex', 'preTestScore', 'postTestScore'])
df

#Fill in missing data with zeros
# df.fillna(0)

#Fill in missing data with bfill/ffill
# df.fillna(method='bfill')

In [ ]:
# Fill in missing in preTestScore with the mean value of preTestScore
df["preTestScore"].fillna(df["preTestScore"].mean(), inplace=True)
df

In [ ]:
df["postTestScore"].fillna(df["postTestScore"].median(), inplace=True)
df

In [ ]:
df["age"].fillna(method='ffill', inplace=True)
df

In [ ]:
df["sex"].fillna(method='bfill', inplace=True)
df

### Exercise 3

Read the [Heart Disease Data Set](https://archive.ics.uci.edu/ml/datasets/Heart+Disease) for 'processed.cleveland.data' and impute the missing values.

In [ ]:
import pandas as pd

heart = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data', header=None, na_values='?')
heart.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num',]

In [ ]:
heart.describe()

In [ ]:
heart.isnull().values.sum()

In [ ]:
heart[heart.isnull().any(axis=1)]

In [ ]:
#impute the missing values
heart["ca"].fillna(heart["ca"].mean(), inplace=True)
heart["thal"].fillna(heart["thal"].mean(), inplace=True)

In [ ]:
nans = lambda heart: heart[heart.isnull().any(axis=1)]
nans(heart)

In [ ]:
heart.iloc[[87, 166, 192, 266, 287,302]]

## Feature Pipelines

With any of the preceding examples, it can quickly become tedious to do the transformations by hand, especially if you wish to string together multiple steps.
For example, we might want a processing pipeline that looks something like this:

1. Impute missing values using the mean
2. Transform features to quadratic
3. Fit a linear regression

To streamline this type of processing pipeline, Scikit-Learn provides a ``Pipeline`` object, which can be used as follows:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

model = make_pipeline(SimpleImputer(strategy='mean'),
                      PolynomialFeatures(degree=2),
                      LinearRegression())

In [ ]:
X = np.array([[ nan, 0,   3  ],
              [ 3,   7,   9  ],
              [ 3,   5,   2  ],
              [ 4,   nan, 6  ],
              [ 8,   8,   1  ]])
y = np.array([14, 16, -1,  8, -5])

This pipeline looks and acts like a standard Scikit-Learn object, and will apply all the specified steps to any input data.

In [ ]:
model.fit(X, y)  # X with missing values, from above
print(model.predict(X))

All the steps of the model are applied automatically.
Notice that for the simplicity of this demonstration, we've applied the model to the data it was trained on; this is why it was able to perfectly predict the result. However, in most time, we need to fine tune the hyperparameters and perform model validation.

### Exercise 4

Given the following code, use ``pipeline`` to transform to ``PolynomialFeatures`` with degree=2 and fit the ``LinearRegression`` model.

```{python}
import numpy as np
from sklearn.linear_model import LinearRegression

x = np.arange(10)[:, None]
y = np.ravel(x) ** 2

p = np.array([1, 2])
model = LinearRegression().fit(x ** p, y)
model.predict(11 ** p)
```

In [ ]:
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

x = np.arange(10)[:, None]
y = np.ravel(x) ** 2 #flatten up the array

# p = np.array([1, 2])
# model = LinearRegression().fit(x ** p, y)
# model.predict(11 ** p)

pipemodel = make_pipeline(PolynomialFeatures(degree=2),LinearRegression())

pipemodel.fit(x, y)
print(y)
print(pipemodel.predict(x))

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

x = np.arange(10)[:, None]
y = np.ravel(x) ** 2

p = np.array([1, 2])
model = LinearRegression().fit(x ** p, y)
model.predict(x ** p)
print(y)
print(model.predict(x ** p))

## Feature Scaling and Normalization

An alternative approach to Z-score normalization (or standardization) is the so-called Min-Max scaling (often also simply called “normalization” - a common cause for ambiguities).
In this approach, the data is scaled to a fixed range - usually 0 to 1.
The cost of having this bounded range - in contrast to standardization - is that we will end up with smaller standard deviations, which can suppress the effect of outliers.

A Min-Max scaling is typically done via the following equation:

$$X_{norm}=\frac{X-X_{min}}{X_{max} - X_{min}} $$

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/wine_data.csv',header=None,usecols=[0,1,2])
df.columns=['Class label', 'Alcohol', 'Malic acid']
df.head()

In [ ]:
from sklearn import preprocessing

#standard scale transform
std_scale = preprocessing.StandardScaler().fit(df[['Alcohol', 'Malic acid']])
df_std = std_scale.transform(df[['Alcohol', 'Malic acid']])

#min max scale transform
minmax_scale = preprocessing.MinMaxScaler().fit(df[['Alcohol', 'Malic acid']])
df_minmax = minmax_scale.transform(df[['Alcohol', 'Malic acid']])

print('Mean after standardization:\nAlcohol={:.2f}, Malic acid={:.2f}'
      .format(df_std[:,0].mean(), df_std[:,1].mean()))
print('\nStandard deviation after standardization:\nAlcohol={:.2f}, Malic acid={:.2f}'
      .format(df_std[:,0].std(), df_std[:,1].std()))

Plot to show the standardized and normalized data points

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

def plot():
    plt.figure(figsize=(8,6))

    plt.scatter(df['Alcohol'], df['Malic acid'],color='green', label='input scale', alpha=0.5)
    plt.scatter(df_std[:,0], df_std[:,1], color='red', label='Standardized N, mu=0, sigma=1', alpha=0.3)
    plt.scatter(df_minmax[:,0], df_minmax[:,1],color='blue', label='min-max scaled [min=0, max=1]', alpha=0.3)

    plt.title('Alcohol and Malic Acid content of the wine dataset')
    plt.xlabel('Alcohol')
    plt.ylabel('Malic Acid')
    plt.legend(loc='upper left')
    plt.grid()

    plt.tight_layout()

plot()
plt.show();

### Exercise 5

Use the ``MinMaxScaler`` to normalized the following dataframe

In [ ]:
import pandas as pd

s1 = pd.Series([1, 2, 3, 4, 5, 6], index=(range(6)))
s2 = pd.Series([10, 9, 8, 7, 6, 5], index=(range(6)))
df = pd.DataFrame(dict(s1=s1,s2=s2), columns=['s1','s2'])
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#min max scale transform
minmax_scale = MinMaxScaler().fit(df[['s1','s2']])
df_minmax = minmax_scale.transform(df[['s1', 's2']])

print(df_minmax)

def plot():
    plt.figure(figsize=(8,6))

    plt.scatter(df['s1'], df['s2'],color='green', label='input scale', alpha=0.5)
    plt.scatter(df_minmax[:,0], df_minmax[:,1],color='blue', label='min-max scaled [min=0, max=1]', alpha=0.3)

    plt.title('s1 and s2 dataset')
    plt.xlabel('s1')
    plt.ylabel('s2')
    plt.legend(loc='upper right')
    plt.grid()

    plt.tight_layout()

plot()
plt.show();